In [2]:
pip install pywin32


In [1]:
import win32com.client

def convert_doc_to_pdf(doc_path, pdf_path):
    word = win32com.client.Dispatch("Word.Application")
    doc = word.Documents.Open(doc_path)
    doc.SaveAs(pdf_path, FileFormat=17)  # 17 corresponds to PDF format
    doc.Close()
    word.Quit()

convert_doc_to_pdf(r'C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil_doc\Training_Module\0700-01-003 Boiler Description.doc', r'C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Training_Module\0700-01-003 Boiler Description.pdf')


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Word', 'Sorry, we couldn\'t find your file. Was it moved, renamed, or deleted?\r ("C:\\...\\0700-01-003 Boiler Descripti...")', 'wdmain11.chm', 24654, -2146823114), None)

In [ ]:
import win32com.client

def convert_ppt_to_pdf(pptx_path, pdf_path):
    powerpoint = win32com.client.Dispatch("PowerPoint.Application")
    deck = powerpoint.Presentations.Open(pptx_path)
    deck.SaveAs(pdf_path, FileFormat=32)
    deck.Close()
    powerpoint.Quit()

convert_ppt_to_pdf('example.pptx', 'output.pdf')


In [7]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
from docx import Document
from pptx import Presentation
import win32com.client 
import comtypes.client  
from pdfminer.high_level import extract_text

def convert_to_pdf(file_path):
    if file_path.endswith('.doc') or file_path.endswith('.docx'):
        word = win32com.client.Dispatch("Word.Application")
        doc = word.Documents.Open(file_path)
        pdf_path = file_path.replace('.docx', '.pdf').replace('.doc', '.pdf')
        doc.SaveAs(pdf_path, FileFormat=17)  
        doc.Close()
        word.Quit()
        return pdf_path
    elif file_path.endswith('.pptx'):
        powerpoint = comtypes.client.CreateObject("PowerPoint.Application")
        presentation = powerpoint.Presentations.Open(file_path)
        pdf_path = file_path.replace('.pptx', '.pdf')
        presentation.SaveAs(pdf_path, FileFormat=32) 
        presentation.Close()
        powerpoint.Quit()
        return pdf_path
    else:
        return None

def count_pages_and_check_figures_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    page_count = len(reader.pages)
    
    text = extract_text(pdf_path).lower()
    figure_pages = []
    figure_count = 0
    table_pages = []
    table_count = 0
    thai_language_pages = []
    
    for i, page in enumerate(reader.pages):
        page_text = page.extract_text().lower()
        
        # Check for figures
        if any(keyword in page_text for keyword in ['figure', 'fig.']):
            figure_pages.append(i + 1)
            figure_count += 1
        
        # Check for tables
        if 'table' in page_text or re.search(r'\btable\b', page_text):
            table_pages.append(i + 1)
            table_count += 1
        
        # Check for Thai language
        if re.search(r'[\u0E00-\u0E7F]', page_text):  
            thai_language_pages.append(i + 1)
    
    has_figures = figure_count > 0
    has_tables = table_count > 0
    has_thai_language = len(thai_language_pages) > 0
    
    return page_count, has_figures, figure_count, figure_pages, has_tables, table_count, table_pages, has_thai_language, thai_language_pages

def process_files_in_subfolders(root_folder):
    results = []
    total_files = 0
    
    for subdir, _, files in os.walk(root_folder):
        total_files += len(files)
    
    processed_files = 0
    
    for subdir, _, files in os.walk(root_folder):
        subfolder_name = os.path.basename(subdir)  
        for file in files:
            file_path = os.path.join(subdir, file)
            processed_files += 1
            print(f"Processing {file} ({processed_files}/{total_files})...")
            
            # Convert non-PDF files to PDF first
            if file.endswith('.pdf'):
                pdf_path = file_path
            elif file.endswith('.doc') or file.endswith('.docx') or file.endswith('.pptx'):
                pdf_path = convert_to_pdf(file_path)
                if not pdf_path:
                    print(f"Skipping {file} (Conversion to PDF failed)")
                    continue  
            else:
                print(f"Skipping {file} (Unsupported file type)")
                continue  
            
            # Analyze 
            page_count, has_figures, figure_count, figure_pages, has_tables, table_count, table_pages, has_thai_language, thai_language_pages = count_pages_and_check_figures_pdf(pdf_path)
            file_type = 'PDF' if file.endswith('.pdf') else 'Converted to PDF'
            
            results.append({
                'file_name': file,
                'subfolder_name': subfolder_name,
                'file_type': file_type,
                'page_count': page_count,
                'has_figures': has_figures,
                'figure_count': figure_count,
                'figure_pages': figure_pages,
                'has_tables': has_tables,
                'table_count': table_count,
                'table_pages': table_pages,
                'has_thai_language': has_thai_language,
                'thai_language_pages': thai_language_pages
            })

    return results

def main():
    root_folder = r'C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc'  # Replace with your root folder path

    print(f"Starting the document analysis in {root_folder}")
    file_results = process_files_in_subfolders(root_folder)
    
    # Convert the results to a DataFrame
    df = pd.DataFrame(file_results)
    
    # Display the DataFrame
    print(df)
    
    # Save the DataFrame to a CSV file
    df.to_csv('results_analysis_pdf.csv', index=False)
    print(f"Analysis complete! Results saved to 'results_analysis.csv'.")

if __name__ == "__main__":
    main()


Starting the document analysis in C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc
Processing API 1004_120.40-095.pdf (1/36)...
Processing ASME B16.5-2003 Pipe Flanges-Flanged Fittings NPS 12 Through NPS24 MetricInch Standard_120.70-082.pdf (2/36)...
Processing ASTM Cetane Method For Rating Diesel Fuels 1963_120.80-006.pdf (3/36)...
Processing IEC 112_120.180-158.pdf (4/36)...
Processing R1713001-M7008-24-8301-1502-REV0.pdf (5/36)...
Processing R1814001-M0020-75-0001-0084-rev1.pdf (6/36)...
Processing EN-QTD-45_Integrity Operating Windows Control Plan for U-7500.docx (7/36)...
Processing Enqpr04_Integrity Operating Windows Procedure.pdf (8/36)...
Processing Enqpr05_Spare Part Management Procedure.pdf (9/36)...
Processing MPOF-QWI-750101_START UP SHELL SULFOLANE UNIT (U-7500) WORK INSTRUCTION.docx (10/36)...
Processing MPOF-QWI-750201 NORMAL SHUTDOWN SULFOLANE UNIT U7500 WORK INSTRUCTION.docx (11/36)...
Processing Mpofqpr03_Product Storag